In [2]:
from bs4 import BeautifulSoup
import requests

# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Importar librerías para procesamiento de texto
# -----------------------------------------------------------------------
import re

# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode

In [ ]:

def tabla_oscars(url):

    respuesta_oscars= requests.get(url)

    sopa_oscars=BeautifulSoup(respuesta_oscars.content, 'html.parser')

    tabla_premios=sopa_oscars.find('table', {'class': 'wikitable'})

    cada_ceremonia= tabla_premios.find_all('tr')

    ceremonias=[]

    for fila in cada_ceremonia[1:]:
        
        celdas = fila.find_all('td')
        ceremonia = celdas[1].text.strip()
        
        año_ceremonia = re.search(r'\d{4}', ceremonia)   
        año = int(año_ceremonia.group())  
        
        pelicula = celdas[2].text.strip()
        
        director = re.sub(r'\([^)]*\)', '', celdas[3].text.strip())
        
        actor = re.sub(r'\([^)]*\)', '', celdas[4].text.strip())
        
        actriz = re.sub(r'\([^)]*\)', '', celdas[5].text.strip())
        
        actor_reparto= re.sub(r'\([^)]*\)', '', celdas[6].text.strip())
        
        actriz_reparto=re.sub(r'\([^)]*\)', '', celdas[7].text.strip())
        
        tupla_ceremonia = (año, pelicula, director, actor, actriz, actor_reparto, actriz_reparto)
        
        ceremonias.append(tupla_ceremonia)

    return ceremonias

    

In [ ]:
tabla_wikipedia=tabla_oscars("https://es.wikipedia.org/wiki/Premios_%C3%93scar")

In [ ]:
df_oscars = pd.DataFrame(tabla_wikipedia, columns=["Ceremonia", "Mejor película", "Mejor director", "Mejor actor", "Mejor actriz", "Mejor actor de reparto", "Mejor actriz de reparto"])
df_oscars

In [19]:
df_oscars.to_csv ("tabla_oscars.csv", index=False)

In [ ]:
#Cuando volvemos a abrir el CSV sustituimos los NaN por "Sin datos" para evitar errores en la carga en la BBDD de SQL

df_oscars= pd.read_csv('tabla_oscars.csv')
df_oscars.fillna("Sin datos", inplace=True)

In [ ]:
lista_oscars= [tuple(x) for x in df_oscars.values]

In [ ]:

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='localhost', database='BHO')


mycursor = cnx.cursor()
sql = "INSERT IGNORE INTO oscars (ceremonia, mejor_pelicula, mejor_director, mejor_actor, mejor_actriz, mejor_actor_reparto, mejor_actriz_reparto )VALUES (%s, %s,%s, %s,%s, %s,%s)"

try: 
    for fila in lista_oscars: 
        mycursor.execute(sql,fila)
        cnx.commit()
        print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

cnx.close()